# Facility Fit Tester
### Pick estimates of unit operations or select from type drop down to autopopulate

In [11]:
# from platform import python_version 

# python_version()

#!pip freeze>requirements.txt

In [13]:
# import pkg_resources
# import types
# def get_imports():
#     for name, val in globals().items():
#         if isinstance(val, types.ModuleType):
#             # Split ensures you get root package, 
#             # not just imported function
#             name = val.__name__.split(".")[0]

#         elif isinstance(val, type):
#             name = val.__module__.split(".")[0]

#         # Some packages are weird and have different
#         # imported names vs. system/pip names. Unfortunately,
#         # there is no systematic way to get pip names from
#         # a package's imported name. You'll have to add
#         # exceptions to this list manually!
#         poorly_named_packages = {
#             "PIL": "Pillow",
#             "sklearn": "scikit-learn"
#         }
#         if name in poorly_named_packages.keys():
#             name = poorly_named_packages[name]

#         yield name
# imports = list(set(get_imports()))

# # The only way I found to get the version of the root package
# # from only the name of the package is to cross-check the names 
# # of installed packages vs. imported packages
# requirements = []
# for m in pkg_resources.working_set:
#     if m.project_name in imports and m.project_name!="pip":
#         requirements.append((m.project_name, m.version))

# for r in requirements:
#     print("{}=={}".format(*r))

In [3]:

import sys 
from ipywidgets import HBox, VBox, Text, Button, Layout, GridBox, Dropdown,RadioButtons,AppLayout,GridspecLayout,Label
from ipywidgets import *
import numpy as np
import pandas as pd
from datetime import datetime
import requests
import json
import requests
import datetime
import webbrowser
import time
from scipy import stats
from ipysheet import sheet, cell, row, column, cell_range
import ipysheet
import ipyaggrid
import qgrid

In [8]:

#make dictionary of states and codes 
Types=['Blank','mAb','Recombinant virus','Phage']



class NOAA_APP2():
    def __init__(self):
        #initialize buttons 
        layout={'border': '1px solid black'} 
        style = {'description_width': '200px',}
        layout = {'width:':'4000px'}
        self.docButton= widgets.Button(description='URL Tester',style=style)
        self.Process=widgets.Dropdown(options=Types,description='Process',layout = {'width:':'200px'})
        
        self.BRX=widgets.Dropdown(options=range(0,5),description='Seed BRXs',layout = {'width:':'200px'})
        self.Harvest=widgets.Dropdown(options=['NA','Ksep','Disk'],description='Harvest',layout = {'width:':'200px'})
        self.ViralF=widgets.Dropdown(options=range(0,2),description='Viral Filtration',layout = {'width:':'200px'})
        self.ViralI=widgets.Dropdown(options=range(0,2),description='Viral Inactivation',layout = {'width:':'200px'})
        self.Pr = widgets.Dropdown(options=range(0,5),description='Prod BRXs',layout = {'width:':'200px'})
        self.TFF=widgets.Dropdown(options=range(0,2),description='TFF',layout = {'width:':'200px'})
        self.Chrom=widgets.Dropdown(options=range(0,4),description='# ChromCol',layout = {'width:':'200px'})
        self.Bulk=widgets.Dropdown(options=range(0,4),description='DS Fill',layout = {'width:':'200px'})
        self.LoadProcess = widgets.Button(description="Load Process",style=style, layout=layout)
        self.ClearSheet = widgets.Button(description="Clear Sheet",style=style, layout=layout)
        self.ClearForm = widgets.Button(description="Clear Form",style=style, layout=layout)
        self.radio = widgets.RadioButtons(options=['Date Range','Single Date'],value='Date Range',layout = {'width:':'200px'})
        #start and end default to dates i know have data 
       # self.STARTDATE=widgets.DatePicker( description='Start Date', disabled=False,value=datetime.datetime(2019,5,23))
      #  self.ENDDATE=widgets.DatePicker( description='End Date', value=datetime.datetime(2019,6,1),disabled=False)
        
        import ipysheet
        self.sheet = ipysheet.sheet(rows=50);self.sheet.column_headers=['Unit Op','Working Vol','Flow Rate','pH']
        #row = ipysheet.row(0,['Unit Op'])
        #column = ipysheet.column(1, ["a", "b", "c", "d"], row_start=1, background_color="green")
        self.years = widgets.Dropdown(options=np.linspace(1,5,5),description='Years');self.years.layout.visibility = 'hidden'
        #output widgets 
        self.out =widgets.Output(layout=layout)
        self.out2 =widgets.Output( layout=layout)
        self.out3 = widgets.Output(layout=layout);self.out4=widgets.Output(layout=layout)
        self.about = widgets.Output(layout=layout)
       
        self.status= widgets.Label(style={'description_width': '1000px'})
        # add tabs 
        self.tab = Tab(layout=Layout(height='700px'))
        self.tab.children = [self.about,self.out,self.out2,self.out3,self.out4]
        #tab names 
        tabs = ['About App','Stations Map','Stations Data','Weather Data','Weather Table']
        #for each tab have this header 
        for x in range(0,len(tabs)): 
            self.tab.set_title(x,tabs[x])
            
        #self.radio,self.years,
        self.header=VBox([HBox([self.Process,self.LoadProcess]),
                          VBox([self.BRX,self.Pr,self.ViralF,self.ViralI,self.Chrom,self.TFF,self.Bulk]),
                          HBox([self.ClearSheet,self.ClearForm,self.docButton,self.status])]) 
        self.container=VBox([self.header,self.sheet])#,self.tab])
        #description of app use 
        with self.about:
            print('Pick estimates of unit operations or select from type drop down to autopopulate')
    
              #   'See NOAA documentation button for weather type abbreviations'+'\033[1m \n\n\nQUERY HISTORY\n\n \033[0m' )
    def on_click(self, function): #get data button
        self.LoadProcess.on_click(self.get_data)
        
    def ClearF(self,function):
        self.Process.value ='Blank'
        self.BRX.value = 0;  self.Pr.value = 0;    self.ViralF.value = 0 ;    self.ViralI.value = 0 
        self.TFF.value= 0;       self.Bulk.value = 0;            self.Chrom.value = 0
        
    def on_click_doc(self,button): ##documentation on GHCND 
        DOC='https://www1.ncdc.noaa.gov/pub/data/cdo/documentation/GHCND_documentation.pdf'
        webbrowser.open(DOC)


#Types=['mAb','Recombinant virus','Phage']
    def Process_called(self,function): #radio button for single date or range 
        if self.Process.value == 'mAb':
            self.BRX.value = 3;  self.Pr.value = 1;    self.ViralF.value = 1 ;    self.ViralI.value = 1 
            self.TFF.value= 1;       self.Bulk.value = 1;            self.Chrom.value = 2
            a = [];b =1
            for i in range(0,self.BRX.value):
                a.append('Seed BRX '+str(b))
                b = b+1 
            b = 1
            for i in range(0,self.Pr.value): 
                a.append('Production BRX ' +str(b))
                b = b+1
            b = 1
            #for i in range(0,len(self.Harvest.value)): 
            a.append('Harvest ' +self.Harvest.value)
            for i in range(0,self.Chrom.value): 
                a.append('Chromatography ' +str(b))
                b = b+1         
            b = 1
            column = ipysheet.column(0, a, row_start=0)
           # self.sheet(rows=b)
        else:
            self.BRX.value = 3
            self.Chrom.value = 3
            self.ViralI.value = 0 
     
    def get_data(self,function):
        self.status.value='2'


   
    
    def get_layout(self):
        return self.container
setup = NOAA_APP2()

In [9]:
def on_setup(btn): #setup tab layout 
    setup.get_data()
#query button, documention button, radio button    
setup.LoadProcess.on_click(setup.on_click)
setup.docButton.on_click(setup.on_click_doc)
setup.Process.observe(setup.Process_called)
setup.ClearForm.on_click(setup.ClearF)
setup.get_layout()


In [ ]:
#         #get statsions 
#         self.status.value='Retreiving stations...please wait'
#         df = self.get_stations()
#          #clear output
#         self.out.clear_output(); self.out2.clear_output(); self.out3.clear_output(); self.out4.clear_output()
#         if len(df)!=0: #if we return stations
#             self.status.value='Stations found'
#             with self.out: #show map of stations 
#                 fig = go.Figure(data=go.Scattergeo(lon = df['longitude'], lat = df['latitude'],
#                         text = df['name'],mode = 'markers',         ))
#                 fig.update_layout(title = str(self.Process.value)+'Stations with Data',geo_scope='usa' ); display(fig)
#                 self.tab.children=list(self.tab.children)
#             with self.out2: #print dataframe to tab of weather stations 
#                 s=ipysheet.from_dataframe(df)
#                 display(df)
#             self.status.value="Retrieved "+str(len(df))+" stations from NOAA within date range"
#             self.stationList =  df['id']
#             self.status.value='getting weather....please wait'
#             w= self.get_weather(self.stationList)
#             #if there is weather data 
#             if len(w)!=0:        
#                 self.status.value="Successfully retrieved "+str(len(w))+" weather records"   
#                 with self.out3: #scatter by type 
#                     fig2 = go.Figure(data=px.scatter(w,x='date',y='value',color='datatype' ))
#                     fig2.update_layout(title = 'Weather Data',xaxis_tickformat = '%d %B <br> %Y' ); display(fig2)
#                     display(w.groupby('datatype').describe())
#                 with self.out4:
#                     sheet=ipysheet.from_dataframe(w)
#                     display(sheet)
#                     self.tab.children = list(self.tab.children)
#             else:
#                 self.status.value='No weather for date range'
